In [1]:
""" Analyse the HVS survey data cross-matched with Gaia and Galex. """

##### Dependencies
import numpy as np
%matplotlib osx
import matplotlib.pyplot as plt
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord as coord
import copy
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score
from matplotlib.colors import LogNorm
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')

In [2]:
##### Options
simplifyclasses = True
if simplifyclasses == True:
    type_unique = [-1,0,1]
    type_text = ['QSO','B/BHB','WD']
    type_colors = ['limegreen','deepskyblue','orangered']
else:
    type_unique = [-2,-1,0,1,2,3,7]
    type_text = ['cont','QSO','B/BHB','DA','DZ','DB','Weird']
    type_colors = ['limegreen','greenyellow','deepskyblue','orangered','red','darkred','darkorchid']

In [3]:
def cleandict(DICT,STRING):
    ### Remove all columns in DICT which start with string.
    nSTRING = len(STRING)
    copyDICT = copy.deepcopy(DICT)
    for k,v in DICT.items():
        if k[:nSTRING] == STRING:
            del copyDICT[k]
    return copyDICT

In [4]:
##### Load in HVS data
hvsdata = np.load('/Users/douglasboubert/Documents/Science/GaiaDR2/HVSsurvey/data/gagahvs.npz')
locals().update(hvsdata)
gagahvsbox=gagahvsbox.item()

##### Simplify classes
nhvs = gagahvsbox['ra'].shape[0]
HVSCLASS = gagahvsbox['hvswd']

if simplifyclasses:
    HVSCLASS[HVSCLASS ==  2] =  1 # DZ -> DA
    HVSCLASS[HVSCLASS ==  3] =  1 # DB -> DA
    HVSCLASS[HVSCLASS == -2] = -1 # cont -> QSO
    HVSCLASS[HVSCLASS ==  7] =  0 # Weird -> B/BHB
    
gagahvsbox['CLASS'] = HVSCLASS
gagahvsbox['u_g0'] = gagahvsbox['hvsug0']
gagahvsbox['g_r0'] = gagahvsbox['hvsgr0']
#gagahvsbox['r_i0'] = gagahvsbox['hvsri0']

gagahvsbox = cleandict(gagahvsbox,'hvs')

In [5]:
##### Load in BHB data
bhbdata = np.load('/Users/douglasboubert/Documents/Science/GaiaDR2/HVSsurvey/data/gagabhb.npz')
locals().update(bhbdata)
gagabhbbox=gagabhbbox.item()

##### Simplify classes
nbhb = gagabhbbox['ra'].shape[0]
BHBCLASS = 0.0*np.ones(nbhb)
    
gagabhbbox['CLASS'] = BHBCLASS
gagabhbbox['u_g0'] = gagabhbbox['bhbu-g']
gagabhbbox['g_r0'] = gagabhbbox['bhbg-r']

gagabhbbox = cleandict(gagabhbbox,'bhb')

In [6]:
##### Load in WD data
wddata = np.load('/Users/douglasboubert/Documents/Science/GaiaDR2/HVSsurvey/data/gagawd.npz')
locals().update(wddata)
gagawdbox=gagawdbox.item()

##### Simplify classes
nwd = gagawdbox['ra'].shape[0]
wdclass_string = gagawdbox['wdCLASS']
wdclass = np.array([wdclass_string[i][-1] for i in range(nwd)])
WDCLASS = np.zeros(nwd)

WDTYPES = ['A','B','C','O','Q','X','Z']
if simplifyclasses:
    WDCLASSIFICATIONS = [1,1,1,1,1,1,1]
else:
    WDCLASSIFICATIONS = [1,3,7,7,7,7,2]

for wdt,wdc in zip(WDTYPES,WDCLASSIFICATIONS):
    WDCLASS[wdclass ==  wdt] =  wdc
    
gagawdbox['CLASS'] = WDCLASS
gagawdbox['u_g0'] = gagawdbox['wdUMAG']-gagawdbox['wdGMAG']
gagawdbox['g_r0'] = gagawdbox['wdGMAG']-gagawdbox['wdRMAG']

gagawdbox = cleandict(gagawdbox,'wd')

In [7]:
##### Load in QSO data
qsodata = np.load('/Users/douglasboubert/Documents/Science/GaiaDR2/HVSsurvey/data/gagaqso.npz')
locals().update(qsodata)
gagaqsobox=gagaqsobox.item()

##### Simplify classes
nqso = gagaqsobox['ra'].shape[0]
QSOCLASS = -1.0*np.ones(nqso)
    
gagaqsobox['CLASS'] = QSOCLASS
gagaqsobox['u_g0'] = gagaqsobox['qsoug0']
gagaqsobox['g_r0'] = gagaqsobox['qsogr0']

gagaqsobox = cleandict(gagaqsobox,'qso')

In [8]:
##### Form merged dict
randomorder = np.random.choice(range(nhvs+nbhb+nwd+nqso),nhvs+nbhb+nwd+nqso,replace=False)
gagabox = {k:np.hstack([gagahvsbox[k],gagabhbbox[k],gagawdbox[k],gagaqsobox[k]])[randomorder] for k in gagahvsbox.keys()}
N = gagabox['ra'].shape[0]

### Correct FUV mag
gagabox['fuv_mag'] = np.array([float(fuv) if fuv != 'None' else np.nan for fuv in gagabox['fuv_mag']])
gagabox['fuv_magerr'] = np.array([float(fuv) if fuv != 'None' else np.nan for fuv in gagabox['fuv_magerr']])

In [9]:
hasgaiacrossmatch = np.where(np.isnan(gagabox['phot_g_mean_mag'])==False)
hasgaiacolorcrossmatch = np.where(np.isnan(gagabox['phot_bp_mean_mag'])==False)
hasgaiaastcolorcrossmatch = np.where((np.isnan(gagabox['phot_bp_mean_mag'])==False) & (np.isnan(gagabox['parallax'])==False))
hasgalexcrossmatch = np.where(np.isnan(gagabox['nuv_mag'])==False)
hasallwisecrossmatch = np.where(np.isnan(gagabox['w1mpro'])==False)
hasgaalastcolorcrossmatch = np.where((np.isnan(gagabox['phot_bp_mean_mag'])==False) & (np.isnan(gagabox['w1mpro'])==False) & (np.isnan(gagabox['w2mpro'])==False) & (np.isnan(gagabox['parallax'])==False))
hasgagacrossmatch = np.where((np.isnan(gagabox['nuv_mag'])==False)&(np.isnan(gagabox['phot_g_mean_mag'])==False))
hasgagacolorcrossmatch = np.where((np.isnan(gagabox['nuv_mag'])==False)&(np.isnan(gagabox['phot_bp_mean_mag'])==False))
hasgagaastcrossmatch = np.where((np.isnan(gagabox['nuv_mag'])==False)&(np.isnan(gagabox['parallax'])==False))
hasgagaastcolorcrossmatch = np.where((np.isnan(gagabox['nuv_mag'])==False)&(np.isnan(gagabox['parallax'])==False)&(np.isnan(gagabox['phot_bp_mean_mag'])==False))
print('All stars\t\t\t',gagabox['phot_g_mean_mag'].shape[0])
print("Has Gaia crossmatch\t\t",hasgaiacrossmatch[0].shape[0])
print("Has Gaia color crossmatch\t",hasgaiacolorcrossmatch[0].shape[0])
print("Has Gaia ast color crossmatch\t",hasgaiaastcolorcrossmatch[0].shape[0])
print('Has Galex crossmatch\t\t',hasgalexcrossmatch[0].shape[0])
print('Has Allwise crossmatch\t\t',hasallwisecrossmatch[0].shape[0])
print('Has GaAl ast color crossmatch\t',hasgaalastcolorcrossmatch[0].shape[0])
print('Has GaGa crossmatch\t\t',hasgagacrossmatch[0].shape[0])
print('Has GaGa color crossmatch\t',hasgagacolorcrossmatch[0].shape[0])
print('Has GaGaAst crossmatch\t\t',hasgagaastcrossmatch[0].shape[0])
print('Has GaGaAstColor crossmatch\t',hasgagaastcolorcrossmatch[0].shape[0])

All stars			 28982
Has Gaia crossmatch		 21008
Has Gaia color crossmatch	 19905
Has Gaia ast color crossmatch	 17985
Has Galex crossmatch		 13535
Has Allwise crossmatch		 21019
Has GaAl ast color crossmatch	 14400
Has GaGa crossmatch		 11766
Has GaGa color crossmatch	 11417
Has GaGaAst crossmatch		 10817
Has GaGaAstColor crossmatch	 10768


In [10]:
##### Load in true hypervelocity stars
hvsdata = pd.read_csv('/Users/douglasboubert/Documents/Science/GaiaDR2/HVSsurvey/data/hypervelocitystars18052018.csv')
hvsradec = np.array( [ hvsdata['R.A.'][i].split(',')[0]+hvsdata['Dec.'][i].split(',')[0] for i in range(len(hvsdata['R.A.']))] )
hvscoord = coord(hvsradec,unit=(u.hourangle, u.deg))
hvsra = hvscoord.ra.deg
hvsdec = hvscoord.dec.deg

##### Cross-match true hypervelocity stars with HVS survey
# Small enough that we can do this naively.
dtr = np.pi/180.0
skysep = (3600/dtr)*np.arccos(np.sin(gagabox['dec']*dtr)*np.sin(hvsdec[:,np.newaxis]*dtr)+np.cos(gagabox['dec']*dtr)*np.cos(hvsdec[:,np.newaxis]*dtr)*np.cos((gagabox['ra']-hvsra[:,np.newaxis])*dtr)).T
skysep[np.isnan(skysep)==True]=1e10
minskysep = np.min(skysep,axis=1)
ishvs = np.where(minskysep<1)

##### Check the cross-match worked
# Check that these are all B stars
assert np.array_equal(gagabox['CLASS'][ishvs],np.zeros(ishvs[0].shape[0]))
# Print number found
print('Cross-matched',ishvs[0].shape[0],'out of',hvsra.shape[0])

Cross-matched 27 out of 30


In [11]:
##### Labels
n_type = len(type_unique)
type_index = []
for TYPE in type_unique:
    type_index.append(np.where(gagabox['CLASS']==TYPE))

##### Type plotting
def type_plotting(X,Y,XSTR=None,YSTR=None,PLOT=False,PLOTHVS=False,BOX=gagabox,PLOTTYPES=None):
    if isinstance(X,str):
        U = BOX[X]
        XSTR = X
    else:
        U = X
    if isinstance(Y,str):
        V = BOX[Y]
        YSTR = Y
    else:
        V = Y
        
    if PLOTTYPES == None:
        PLOTTYPES = type_unique
    for i in range(n_type):
        if type_unique[i] in PLOTTYPES:
            plt.scatter(U[type_index[i]],V[type_index[i]],c=type_colors[i],label=type_text[i],alpha=0.5)
    if PLOTHVS == True:
        plt.scatter(U[ishvs],V[ishvs],label='HVS',marker='*',c='k')
    if PLOT == True:
        plt.xlabel(XSTR)
        plt.ylabel(YSTR)
        plt.legend(loc='best')
        plt.show()

In [35]:
##### However astrometry is good enough
total_pm = np.sqrt(gagabox['pmra']**2+gagabox['pmdec']**2.)
type_plotting(np.log10(total_pm),'phot_g_mean_mag',PLOTHVS=True)
#plt.scatter(totalpm[type_index[2]],gagabox['phot_g_mean_mag'][type_index[2]])
plt.show()


In [115]:
##### However astrometry is good enough
total_pm = np.sqrt(gagabox['pmra']**2+gagabox['pmdec']**2.)
type_plotting(np.log10(totalpm),gagabox['w1mpro']-gagabox['w2mpro'],PLOTHVS=True)
#plt.scatter(totalpm[type_index[2]],gagabox['phot_g_mean_mag'][type_index[2]])
plt.show()

In [13]:
type_plotting('bp_g','g_rp',PLOTHVS=True)

In [51]:
### Form new colour space
# Pick class
impute_class = 1

# Inputs
impute_magnitudes = [gagabox['phot_g_mean_mag'],gagabox['phot_bp_mean_mag'],gagabox['phot_rp_mean_mag'],gagabox['fuv_mag'],gagabox['nuv_mag'],gagabox['w1mpro'],gagabox['w2mpro']]
impute_magnitudes_error = [np.ones(N)*0.1,np.ones(N)*0.1,np.ones(N)*0.1,gagabox['fuv_magerr'],gagabox['nuv_magerr'],gagabox['w1sigmpro'],gagabox['w2sigmpro']]
impute_magnitudes_0 = [impute_magnitudes[i]-gagabox['ebv']*impute_A[i] for i in range(impute_N)]
impute_labels = ['g','bp','rp','nuv','fuv','w1','w2']
impute_A = [0.,0.,0.,3.77,3.39,0,0] # GALEX https://arxiv.org/abs/1805.08951
impute_N = len(impute_labels)
impute_Nlim = 20

# Correct for dust



# Samples with all magnitudes
hasallcolors = np.where((np.isnan(gagabox['phot_bp_mean_mag'])==False)&(np.isnan(gagabox['nuv_mag'])==False)&(np.isnan(gagabox['fuv_mag'])==False)&(np.isnan(gagabox['w1mpro'])==False)&(np.isnan(gagabox['w2mpro'])==False))
hasallcolorsclass = np.where((gagabox['CLASS']==impute_class) & (np.isnan(gagabox['phot_bp_mean_mag'])==False)&(np.isnan(gagabox['nuv_mag'])==False)&(np.isnan(gagabox['fuv_mag'])==False)&(np.isnan(gagabox['w1mpro'])==False)&(np.isnan(gagabox['w2mpro'])==False))


# Function to impute
def imputer(MAG1,MAG2):
    TMP = MAG1-MAG2
    for cl in [-1,0,1]:
        clhas = np.where((np.isnan(TMP)==False) & (gagabox['CLASS']==cl))
        clhasnt = np.where((np.isnan(TMP)==True) & (gagabox['CLASS']==cl))
        TMP[clhasnt] = np.median(TMP[clhas])
    return TMP

# Form dictionary of colors
colorbox={}
for I in range(impute_N):
    for J in range(I+1,impute_N):
        implabel = impute_labels[I]+'_'+impute_labels[J]
        colorbox[implabel] = imputer(impute_magnitudes_0[I],impute_magnitudes_0[J])
colorkeys = list(colorbox.keys())
colorkeys.sort()

# Form input to PCA
Nck = len(colorkeys)
Xcolor = np.zeros((N,Nck))
for i in range(Nck):
    Xcolor[:,i] = colorbox[colorkeys[i]]

# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=impute_Nlim, svd_solver='full',whiten=False)
if False:
    Xcolor_pca = pca.fit_transform(Xcolor)
else:
    pca.fit(Xcolor[hasallcolors])
    Xcolor_pca = pca.transform(Xcolor)
#plt.scatter(Xcolor_pca[hasallcolorsclass,0],Xcolor_pca[hasallcolorsclass,1])

In [34]:
pca.explained_variance_/np.sum(pca.explained_variance_)# bhb

array([4.95384622e-01, 4.63075340e-01, 2.15531405e-02, 1.88098294e-02,
       9.13688480e-04, 2.63379697e-04, 7.62003408e-30, 2.16684179e-32,
       8.73754358e-33, 6.84507571e-33, 4.15214048e-33, 1.00669895e-33,
       7.17462266e-34, 4.99753205e-34, 4.36186737e-34, 3.58981437e-34,
       3.02435509e-34, 1.86810415e-34, 1.11757394e-34, 1.13216848e-35])

In [32]:
pca.explained_variance_/np.sum(pca.explained_variance_)# wd

array([7.89060018e-01, 1.76577272e-01, 1.47459724e-02, 1.34107123e-02,
       5.16634904e-03, 1.03967550e-03, 1.88540091e-31, 1.17862000e-32,
       1.03585293e-32, 3.23880499e-33, 2.27342633e-33, 1.07068393e-33,
       7.50839088e-34, 5.13066138e-34, 4.03567960e-34, 3.72230439e-34,
       2.75643380e-34, 2.03091166e-34, 1.74976799e-34, 8.80921915e-35])

In [30]:
pca.explained_variance_/np.sum(pca.explained_variance_)# qso

array([4.73132619e-01, 3.33646561e-01, 1.42598363e-01, 2.68084462e-02,
       1.50344372e-02, 8.77957422e-03, 6.08338023e-30, 1.60641742e-32,
       9.22957018e-33, 3.65484852e-33, 2.68223060e-33, 9.36147766e-34,
       8.10147311e-34, 5.45964354e-34, 5.15191336e-34, 4.21239626e-34,
       3.98986073e-34, 3.56495044e-34, 2.41972674e-34, 2.21469872e-34])

In [37]:
pca.explained_variance_/np.sum(pca.explained_variance_)# all

array([5.87578109e-01, 3.81305919e-01, 1.67224624e-02, 1.06008572e-02,
       3.16179371e-03, 6.30858126e-04, 3.37032050e-31, 1.30267839e-32,
       1.21818297e-32, 3.72916622e-33, 2.80886174e-33, 1.88671456e-33,
       8.63195216e-34, 6.41711369e-34, 4.00453580e-34, 3.46818829e-34,
       2.10064380e-34, 1.78433929e-34, 8.53573819e-35, 4.38501843e-35])

In [53]:
type_plotting(Xcolor_pca[:,0],Xcolor_pca[:,1],PLOT=True)

In [419]:
#### Use marginalisation for imputation!
# Do it in magnitudes to avoid correlations
# Should be analytic!

# Form variable containing dereddened magnitudes and errors
Xmag_mu = np.array(impute_magnitudes_0).T
Xmag_error = np.array(impute_magnitudes_error).T

# Correct NaNs
Xmag_mu[np.isnan(Xmag_mu)==True] = 0.0
Xmag_error[np.isnan(Xmag_error)==True] = 10.
Xmag_var = np.power(Xmag_error,2.0)

# Form colours with covariance matrix
def offdiag_indices(n,k):
    rows, cols = np.indices((n,n))
    return np.diag(rows, k=k), np.diag(cols, k=k)
#Xcol_mu = (Xmag[:,:,np.newaxis]-Xmag[:,np.newaxis,:])[:,np.triu_indices(7,1)[0],np.triu_indices(7,1)[1]]
NXcol = Xcol_mu.shape[1]
Xcol_mu = Xmag_mu-np.roll(Xmag_mu,-1,axis=1)
Xcol_cov = np.zeros((N,NXcol,NXcol))
Xcol_cov[:,np.diag_indices(NXcol)[0],np.diag_indices(NXcol)[1]] = Xmag_var+np.roll(Xmag_var,-1,axis=1)
Xcol_cov[:,offdiag_indices(NXcol,1)[0],offdiag_indices(NXcol,1)[1]] = Xmag_var[:,1:]
Xcol_cov[:,offdiag_indices(NXcol,-1)[0],offdiag_indices(NXcol,-1)[1]] = Xmag_var[:,1:]
Xcol_cov[:,0,NXcol-1] = Xcol_cov[:,NXcol-1,0] = Xmag_var[:,0]



In [420]:
Xmag_mu[19]

array([19.20739937, 19.20890045, 18.88120079,  0.        , 20.23370736,
       16.71500015, 16.48600006])

In [421]:
Xmag_var[19]

array([1.00000000e-02, 1.00000000e-02, 1.00000000e-02, 1.00000000e+02,
       3.30843359e-02, 9.40900056e-03, 7.89609934e-02])

In [422]:
Xcol_mu[19]

array([-1.50108337e-03,  3.27699661e-01,  1.88812008e+01, -2.02337074e+01,
        3.51870720e+00,  2.29000092e-01, -2.72139931e+00])

In [423]:
np.diag(Xcol_cov[19])

array([2.00000000e-02, 2.00000000e-02, 1.00010000e+02, 1.00033084e+02,
       4.24933364e-02, 8.83699940e-02, 8.89609934e-02])

In [426]:
np.dot(Xcol_invcov[19],Xcol_cov[19])[2,2]

0.9999999999966318

In [425]:
# Define function to calculate z_c,mu_c,cov_c
i=19
Xcol_invcov = np.linalg.inv(Xcol_cov)
Xcol_covi = Xcol_cov+Xcol_cov[i]
Xcol_mui = Xcol_mu[i]-Xcol_mu
lnZ_c = -0.5*np.einsum('ni,nij,nj->n',Xcol_mui,np.linalg.inv(Xcol_covi),Xcol_mui)-0.5*np.log(np.linalg.det(2.0*np.pi*Xcol_covi))
cov_c = np.linalg.inv(Xcol_invcov[i]+Xcol_invcov)
mu_c = np.einsum('nij,jk,k->ni',cov_c,Xcol_invcov[i],Xcol_mu[i])+np.einsum('nij,njk,nk->ni',cov_c,Xcol_invcov,Xcol_mu)
noti = np.setdiff1d(range(N),i)
norm_c = np.sum(np.exp(lnZ_c[noti]))
mu_new = np.sum((mu_c*np.exp(lnZ_c[:,np.newaxis]))[noti],axis=0)/norm_c
cov_new = np.sum((cov_c*np.exp(lnZ_c[:,np.newaxis,np.newaxis]))[noti],axis=0)/norm_c

In [471]:
# Define function to calculate z_c,mu_c,cov_c
i=19
Xcol_covi = Xcol_cov+Xcol_cov[i]
Xcol_invcovi = np.linalg.inv(Xcol_covi)
Xcol_mui = Xcol_mu[i]-Xcol_mu
lnZ_c = -0.5*np.einsum('ni,nij,nj->n',Xcol_mui,np.linalg.inv(Xcol_covi),Xcol_mui)-0.5*np.log(np.linalg.det(2.0*np.pi*Xcol_covi))
cov_c = np.einsum('ij,njk,nkl->nil',Xcol_cov[i],Xcol_invcovi,Xcol_cov)
mu_c = np.einsum('ij,njk,nk->ni',Xcol_cov[i],Xcol_invcovi,Xcol_mu)+np.einsum('nij,njk,k->ni',cov_c,Xcol_invcovi,Xcol_mu[i])
#noti = np.setdiff1d(range(N),i)
noti = np.setdiff1d(hasallcolors,i)
from scipy.special import logsumexp
norm_c = logsumexp(lnZ_c[noti])
mu_new = np.sum((mu_c*np.exp(lnZ_c[:,np.newaxis]-norm_c))[noti],axis=0)
cov_new = np.sum((cov_c*np.exp(lnZ_c[:,np.newaxis,np.newaxis]-norm_c))[noti],axis=0)

/Users/douglasboubert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]
/Users/douglasboubert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  
/Users/douglasboubert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in multiply
  


In [472]:
np.argmax((lnZ_c-norm_c)[noti])

3948

In [473]:
mu_new,Xcol_mu[19]

(array([ 0.10892439, -0.12557459,  6.46623582, -0.67618031, -1.81709532,
        -5.8841841 , -0.32901438]),
 array([-1.50108337e-03,  3.27699661e-01,  1.88812008e+01, -2.02337074e+01,
         3.51870720e+00,  2.29000092e-01, -2.72139931e+00]))

In [474]:
Xcol_mu[3948]

array([-2.09999084e-03,  8.62998962e-02,  1.75240002e+01,  0.00000000e+00,
       -1.72700005e+01,  5.64001083e-01, -9.02200699e-01])

In [366]:
Xcol_mu.mean(axis=0)

array([-0.0332556 ,  0.26475234, -1.28971078,  0.11507847,  2.32088157,
        0.33618929, -1.71393529])

In [182]:
nsamp = 10000
a = np.random.normal(1.,0.5,nsamp)
b = np.random.normal(2.,1.0,nsamp)
c = np.random.normal(3.,1.5,nsamp)
ab = a-b
ac = a-c
bc = b-c
plt.scatter(ab,ac)
np.cov(np.vstack([ab,ac,bc]))

array([[ 1.24794473,  0.23509676, -1.01284798],
       [ 0.23509676,  2.57700245,  2.34190569],
       [-1.01284798,  2.34190569,  3.35475367]])

In [177]:
Xcol.shape

(28982, 21)

array([2.75752736e+01, 1.78948380e+01, 7.84791790e-01, 4.97502433e-01,
       1.48384232e-01, 2.96064220e-02, 1.58170477e-29, 6.11352135e-31,
       5.71698096e-31, 1.75011249e-31, 1.31820995e-31, 8.85442624e-32,
       4.05100937e-32, 3.01157690e-32, 1.87934453e-32, 1.62763452e-32,
       9.85840463e-33, 8.37397504e-33, 4.00585578e-33, 2.05790653e-33])

In [112]:
j=0
for i in np.argsort(pca.components_[j])[::-1]:
    print(colorkeys[i],pca.components_[j][i])

g_w2 0.3811623226165086
bp_w2 0.3801481470033788
rp_w2 0.352730363599096
g_w1 0.3191410129365432
bp_w1 0.3181268373234134
rp_w1 0.29070905391913066
fuv_w2 0.2763916208079594
nuv_w2 0.24467436113290617
fuv_w1 0.21437031112799407
nuv_w1 0.18265305145294078
g_nuv 0.13648796148360245
bp_nuv 0.13547378587047254
rp_nuv 0.10805600246618982
g_fuv 0.10477070180854922
bp_fuv 0.10375652619541934
rp_fuv 0.07633874279113644
w1_w2 0.06202130967996537
g_rp 0.028431959017412665
bp_rp 0.027417783404282692
g_bp 0.001014175613129882
nuv_fuv -0.03171725967505331


In [ ]:
type_plotting()

In [149]:
##### Machine learning
# Select clean features

removefeatures = ['source_id','u_g0','g_r0','CLASS','log10_total_pm','ra','dec','bp_rp','g_rp','bp_g']
#subsetofstars = hasgagaastcolorcrossmatch
subsetofstars = hasgaiaastcolorcrossmatch
#subsetofstars = hasgaalastcolorcrossmatch




featurebox = {k:v[subsetofstars] for k, v in gagabox.items()}
y = featurebox['CLASS']

### Add features
hasallwise = np.zeros(N)
hasallwise[hasallwisecrossmatch] = 1.0
hasgalex = np.zeros(N)
hasgalex[hasgalexcrossmatch] = 1.0
reduced_pm = gagabox['phot_g_mean_mag']-5.*5.*np.log10(total_pm)
total_pm_over_error = total_pm**2./np.sqrt((gagabox['pmra']*gagabox['pmra_error'])**2+(gagabox['pmdec']*gagabox['pmra_error'])**2.-gagabox['pmra_pmdec_corr']*gagabox['pmra']*gagabox['pmdec']*gagabox['pmra_error']*gagabox['pmdec_error'])


if True:
    addfeatures_keys = ['total_pm_over_error','reduced_pm','total_pm','log10_total_pm','vt','g_variability','bp_variablity','rp_variability','hasallwise','hasgalex','abs_b']
    addfeatures_values = [total_pm_over_error,reduced_pm,total_pm,np.log10(total_pm),np.abs(totalpm/gagabox['parallax']),np.log10(np.sqrt(gagabox['phot_g_n_obs']/gagabox['phot_g_mean_flux_over_error'])),np.log10(np.sqrt(gagabox['phot_bp_n_obs']/gagabox['phot_bp_mean_flux_over_error'])),np.log10(np.sqrt(gagabox['phot_rp_n_obs']/gagabox['phot_rp_mean_flux_over_error'])),hasallwise,hasgalex,np.abs(gagabox['b'])]
    
    for i in range(len(addfeatures_keys)):
        featurebox[addfeatures_keys[i]] = addfeatures_values[i][subsetofstars]

# Add imputed colors
if True:
    for i in range(impute_Nlim):
        featurebox['impute_color_'+str(i)]=Xcolor_pca[subsetofstars,i].ravel()

removehvsfeatures = True
deletelist = []
import copy
copybox = copy.deepcopy(featurebox)
for k, v in copybox.items():
    try:
        if np.isnan(v).any():
            print('deleting',k)
            deletelist.append(k)
            del featurebox[k]
        elif removehvsfeatures and k[:3]=='hvs':
            print('removing HVS survey feature',k)
            deletelist.append(k)
            del featurebox[k]
    except TypeError:
        print('TypeError on',k)
        deletelist.append('k')
        del featurebox[k]
for RF in removefeatures:
    del featurebox[RF]

deleting astrometric_pseudo_colour
deleting astrometric_pseudo_colour_error
TypeError on phot_variable_flag
deleting a_g_val
deleting e_bp_min_rp_val
deleting radial_velocity
deleting radial_velocity_error
deleting rv_template_teff
deleting rv_template_logg
deleting rv_template_fe_h
deleting galex_ra
deleting galex_dec
deleting fuv_mag
TypeError on fuv_magerr
deleting nuv_mag
deleting nuv_magerr
deleting allwise_ra
deleting allwise_dec
deleting w1mpro
deleting w1sigmpro
deleting w2mpro
deleting w2sigmpro


In [150]:
# Form training and testing sets
from sklearn.feature_extraction import DictVectorizer
from random import shuffle
v = DictVectorizer(sparse=False)
D = [dict(zip(featurebox,t)) for t in zip(*featurebox.values())]
#shuffle(D)
X = v.fit_transform(D)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [88]:
# Train
GB = GradientBoostingClassifier(n_estimators=100)
ET = ExtraTreesClassifier(n_estimators=100)
RF = RandomForestClassifier(n_estimators=100)
#AB = AdaBoostClassifier(n_estimators=500)
#VOT = VotingClassifier(estimators=[('gb', GB), ('et', ET), ('rf', RF)], voting='hard')
CL = [GB,ET,RF]
CL_names = ['Gradient Boosting', 'Extra Trees', 'Random Forest']
CL_score = np.zeros((len(CL),2))
for i, cl in zip(range(len(CL)),CL):
    scores = cross_val_score(cl, X_train, y_train, cv=5,n_jobs=-1)
    #cl.fit(X_train, y_train)
    #CL_score[i] = cl.score(X_test,y_test)
    CL_score[i] = scores.mean(), scores.std()
    print(CL_names[i],CL_score[i])

Gradient Boosting [0.97741159 0.00205139]
Extra Trees [0.9706006  0.00176515]
Random Forest [0.97581345 0.00289398]


Gradient Boosting [0.97815751 0.00215111]
Extra Trees [0.97423625 0.00357049]
Random Forest [0.97591729 0.001703  ]

In [154]:
RF = RandomForestClassifier(n_estimators=500,class_weight='balanced')
#GB = GradientBoostingClassifier(n_estimators=500)
CL_best = RF
CL_best.fit(X_train, y_train)
CL_best.score(X_test,y_test)

0.9705309980539338

In [155]:
binsgrid = np.arange(-1.5,2.0,1.0)
midbins = (binsgrid[1:]+binsgrid[:-1])/2.0
plt.figure(figsize=(6,5.8))
hist = plt.hist2d(y_test,CL_best.predict(X_test),bins=[binsgrid,binsgrid],cmin=1,norm=LogNorm(),cmap=plt.cm.Greys,alpha=0.5)[0]
for i,j in zip(np.where(np.isnan(hist)==False)[0],np.where(np.isnan(hist)==False)[1]):
    plt.text(midbins[i],midbins[j],str(int(hist[i,j])),horizontalalignment='center',verticalalignment='center')

plt.gca().set_xticks(type_unique)
plt.gca().set_xticklabels(type_text)
plt.gca().set_yticks(type_unique)
plt.gca().set_yticklabels(type_text)
#plt.axis('equal')
#plt.colorbar()
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.xlim([-1.5,1.5])
plt.ylim([-1.5,1.5])
plt.tight_layout()
plt.savefig('/Users/douglasboubert/Dropbox/ShowAndTell/GAIAhypervelocity/gaiabstar/result/machinelearning/confusion.pdf')
plt.savefig('/Users/douglasboubert/Dropbox/ShowAndTell/GAIAhypervelocity/gaiabstar/result/machinelearning/confusion.png',dpi=500)
plt.show()

In [144]:
def acquire_col(STR):
    keys = np.array(list(D[0].keys()))
    keys.sort()
    retcolind = np.where(keys==STR)[0][0]
    return retcolind
def misclass(TRUEI,PREDI):
    return np.where((CL_best.predict(X_test)==PREDI) & (y_test==TRUEI))
def misclass_col(TI,PI,COLSTR):
    return X_train[misclass(TI,PI),acquire_col(COLSTR)]

In [143]:
### Misclassified QSOs as BHBs

misclass_qso_bhb = np.where((CL_best.predict(X_test)==0.) & (y_test==-1.))
misclass_qso_qso = np.where((CL_best.predict(X_test)==-1.) & (y_test==-1.))
plt.scatter(np.log10(X_train[:,acquire_col('total_pm')]),X_train[:,acquire_col('impute_color_0')],c=y_train,cmap=plt.cm.jet)
#plt.scatter(np.log10(X_test[misclass_qso_qso,acquire_col('total_pm')]),X_test[misclass_qso_qso,acquire_col('impute_color_0')],c='violet',s=30)
#plt.scatter(np.log10(X_test[misclass_qso_bhb,acquire_col('total_pm')]),X_test[misclass_qso_bhb,acquire_col('impute_color_0')],c='k',s=30)



In [148]:
misclass_col(1,-1,'total_pm')

array([[34.62611804,  3.81387715,  0.92382589,  0.65993441,  4.28232617,
         0.599063  ,  0.41053167,  1.61070612,  6.44400381, 27.81069922,
         3.42687073,  1.46534796,  3.84620067,  6.40230128,  5.54595107,
         1.43464254,  2.50864718,  0.7317415 ,  3.5844266 ,  1.52830824,
         7.85678732,  1.34673505,  2.66322569,  3.35185066,  4.40684354,
         2.22793247,  1.99663372,  2.91365683,  8.02998789,  2.4621277 ,
         3.86020613,  0.95087469,  1.67135356,  3.60713434,  1.08714329,
        17.94263586,  8.60695917,  1.84529333,  1.30304664]])

In [153]:
importances = CL_best.feature_importances_
indices = np.argsort(importances)[::-1]
keys = np.array(list(D[0].keys()))
keys.sort()
for i in indices:
    print(keys[i],importances[i])
pltkeys = keys[indices]
pltimportances = importances[indices]

total_pm 0.09474768590882512
total_pm_over_error 0.09257748990024087
reduced_pm 0.07891054217995815
parallax 0.0748512438486473
parallax_over_error 0.07428365572841636
impute_color_0 0.044215786281774604
phot_bp_mean_mag 0.029794612844414932
phot_bp_mean_flux 0.029782567540689593
pmdec 0.02729894752653744
phot_g_mean_flux_over_error 0.025888668544875478
astrometric_weight_al 0.023864216474081348
g_variability 0.02382342488846023
phot_g_mean_flux 0.02339351352021951
hasallwise 0.023286901062807404
impute_color_4 0.02194742913986881
phot_g_mean_mag 0.02181749019498849
phot_bp_rp_excess_factor 0.021655694403593044
impute_color_1 0.02133605417102638
phot_rp_mean_flux 0.0205433421701089
phot_rp_mean_mag 0.020150402528989354
pmra 0.018257903605348003
impute_color_3 0.017058704933738487
phot_bp_mean_flux_over_error 0.014327803431748358
vt 0.011881826548800264
bp_variablity 0.010589684601619529
frame_rotator_object_type 0.010357455094112797
dec_error 0.009903067055987149
pmdec_error 0.00961721

In [131]:
nbest = 30
bins = np.arange(-0.5,nbest-0.4,1.0)
plt.hist(range(nbest),weights=pltimportances[:nbest],bins=bins,facecolor='None', alpha = 1.0, edgecolor= 'k',lw=1)
plt.xticks(range(nbest), pltkeys[:nbest], rotation='vertical')
# Pad margins so that markers don't get clipped by the axes
plt.margins(0.5)
# Tweak spacing to prevent clipping of tick-labels
plt.subplots_adjust(bottom=0.5)
plt.xlim([-0.5,nbest-0.5])
plt.savefig('/Users/douglasboubert/Dropbox/ShowAndTell/GAIAhypervelocity/gaiabstar/result/machinelearning/importance.pdf')
plt.savefig('/Users/douglasboubert/Dropbox/ShowAndTell/GAIAhypervelocity/gaiabstar/result/machinelearning/importance.png',dpi=500)
plt.show()

In [ ]:
##### Simple plots
type_plotting(np.log10(totalpm),gagabox['bp_rp'],PLOT=True,PLOTHVS=True,XSTR='log10(Total PM) (mas/yr)',YSTR='BP-RP')

In [ ]:
for wdt in WDTYPES:
    pltsub = np.where(np.array(wdclass) ==  wdt)
    plt.scatter(gagawdbox['u_g0'][pltsub],gagawdbox['g_r0'][pltsub],label=wdt)
plt.legend(loc='best')

In [ ]:
pltsub